<a href="https://colab.research.google.com/github/uzair011/random-codes/blob/main/Quant_Researcher_Internship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime

# display settings
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)


In [4]:
#load CSV
PATH_TO_CSV = "/content/wintermute_transfers_search_default_2025-04-08.csv"
df = pd.read_csv(PATH_TO_CSV)
df.head()


,timestamp,from_address,from_entity,to_address,to_entity,token,value,usd,tx_hash,chain,block_number
0,2025-04-08 12:49:57+00:00,0xB1026b8e7276e7AC75410F1fcbbe21796e8f7526,Camelot,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,USDC,438.108563,438.108563,0x98058c529466064c355bdfea3cfebe399344678f20bb...,arbitrum_one,324218955.0
1,2025-04-08 12:49:54+00:00,0xcDa53B1F66614552F834cEeF361A8D12a0B8DaD8,Uniswap,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,ARB,3837.138536,1062.879700,0x405b146ac6a52b93e5a77760a4b49a87015b5c5e0659...,arbitrum_one,324218945.0
2,2025-04-08 12:49:51+00:00,0xb2cc224c1c9feE385f8ad6a55b4d94E92359DC59,Aerodrome Finance,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,WETH,7.345922,11533.612401,0x7f370fc61bed7241d846c4907a4e616e69ce60ff0192...,base,28663622.0
3,2025-04-08 12:49:51+00:00,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,0x72AB388E2E2F6FaceF59E3C3FA2C4E29011c2D38,PancakeSwap,USDC,2143.712587,2143.712587,0x88b4fb948e95cc2b9a22c48efe3d6a6788209302e72f...,base,28663622.0
4,2025-04-08 12:49:50+00:00,0x641C00A822e8b671738d32a431a4Fb6074E5c79d,Uniswap,0x51C72848c68a965f66FA7a88855F9f7784502a7F,Wintermute,WETH,0.470011,737.550466,0x39fc603c882c2f5af51b7f8cfc8eb58753f5e42eff94...,arbitrum_one,324218918.0


In [5]:
import pandas as pd

# cleaning and normalising..
# Convert timestamps and clean strings

# Convert timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Standardize token symbols
df['token'] = df['token'].str.upper().fillna('UNKNOWN')

# Standardize entity strings (important for classification)
df['from_entity'] = df['from_entity'].fillna('').str.upper()
df['to_entity']   = df['to_entity'].fillna('').str.upper()

# Make sure value is numeric
df['value'] = pd.to_numeric(df['value'], errors='coerce').fillna(0)

# USD notional
df['usd'] = pd.to_numeric(df['usd'], errors='coerce').fillna(0)

df.head()

,timestamp,from_address,from_entity,to_address,to_entity,token,value,usd,tx_hash,chain,block_number
0,2025-04-08 12:49:57+00:00,0xB1026b8e7276e7AC75410F1fcbbe21796e8f7526,CAMELOT,0x51C72848c68a965f66FA7a88855F9f7784502a7F,WINTERMUTE,USDC,438.108563,438.108563,0x98058c529466064c355bdfea3cfebe399344678f20bb...,arbitrum_one,324218955.0
1,2025-04-08 12:49:54+00:00,0xcDa53B1F66614552F834cEeF361A8D12a0B8DaD8,UNISWAP,0x51C72848c68a965f66FA7a88855F9f7784502a7F,WINTERMUTE,ARB,3837.138536,1062.879700,0x405b146ac6a52b93e5a77760a4b49a87015b5c5e0659...,arbitrum_one,324218945.0
2,2025-04-08 12:49:51+00:00,0xb2cc224c1c9feE385f8ad6a55b4d94E92359DC59,AERODROME FINANCE,0x51C72848c68a965f66FA7a88855F9f7784502a7F,WINTERMUTE,WETH,7.345922,11533.612401,0x7f370fc61bed7241d846c4907a4e616e69ce60ff0192...,base,28663622.0
3,2025-04-08 12:49:51+00:00,0x51C72848c68a965f66FA7a88855F9f7784502a7F,WINTERMUTE,0x72AB388E2E2F6FaceF59E3C3FA2C4E29011c2D38,PANCAKESWAP,USDC,2143.712587,2143.712587,0x88b4fb948e95cc2b9a22c48efe3d6a6788209302e72f...,base,28663622.0
4,2025-04-08 12:49:50+00:00,0x641C00A822e8b671738d32a431a4Fb6074E5c79d,UNISWAP,0x51C72848c68a965f66FA7a88855F9f7784502a7F,WINTERMUTE,WETH,0.470011,737.550466,0x39fc603c882c2f5af51b7f8cfc8eb58753f5e42eff94...,arbitrum_one,324218918.0


In [6]:
# Identify Counterparty

df['from_entity'].value_counts().head(10), df['to_entity'].value_counts().head(10)

(from_entity
 WINTERMUTE                   231737
 ORCA                          45336
 RAYDIUM                       44271
                               33300
 UNISWAP                       25755
 METEORA (PREV. MERCURIAL)     18964
 PANCAKESWAP                   10801
 FLUID (INSTADAPP)              7104
 CAMELOT                        5361
 AERODROME FINANCE              4503
 Name: count, dtype: int64,
 to_entity
 WINTERMUTE                   184897
 JITO                          48343
 ORCA                          45346
 RAYDIUM                       44262
                               29859
 UNISWAP                       25722
 METEORA (PREV. MERCURIAL)     18959
 PANCAKESWAP                   10807
 FLUID (INSTADAPP)              6829
 CAMELOT                        5368
 Name: count, dtype: int64)

In [7]:
# Define counterparty (very important for classification)

def get_counterparty(row):
    if 'WINTERMUTE' in row['from_entity']:
        return row['to_entity']
    elif 'WINTERMUTE' in row['to_entity']:
        return row['from_entity']
    else:
        # If neither side is labeled Wintermute, treat it as unknown
        return row['to_entity'] if row['to_entity'] != '' else row['from_entity']

df['counterparty'] = df.apply(get_counterparty, axis=1)
df['counterparty'] = df['counterparty'].fillna('').str.upper()

df[['from_entity','to_entity','counterparty']].head(10)


,from_entity,to_entity,counterparty
0,CAMELOT,WINTERMUTE,CAMELOT
1,UNISWAP,WINTERMUTE,UNISWAP
2,AERODROME FINANCE,WINTERMUTE,AERODROME FINANCE
3,WINTERMUTE,PANCAKESWAP,PANCAKESWAP
4,UNISWAP,WINTERMUTE,UNISWAP
5,WINTERMUTE,UNISWAP,UNISWAP
6,CAMELOT,WINTERMUTE,CAMELOT
7,PANCAKESWAP,WINTERMUTE,PANCAKESWAP
8,UNISWAP,WINTERMUTE,UNISWAP
9,PANCAKESWAP,WINTERMUTE,PANCAKESWAP


In [8]:
df['counterparty'].value_counts().head(20)

,count
counterparty,
ORCA,90682
RAYDIUM,88533
UNISWAP,51477
JITO,48343
,42515
METEORA (PREV. MERCURIAL),37923
PANCAKESWAP,21608
FLUID (INSTADAPP),13933
CAMELOT,10729


This output shows that your counterparties are overwhelmingly DeFi protocols:

In [9]:
# Step 3: classification mapping based on your displayed data

mapping = {
    'ORCA': ('DeFi', 'DEX'),
    'RAYDIUM': ('DeFi', 'DEX'),
    'UNISWAP': ('DeFi', 'DEX'),
    'JITO': ('DeFi', 'Staking/MEV'),
    'METEORA (PREV. MERCURIAL)': ('DeFi', 'Liquidity/AMM'),
    'PANCAKESWAP': ('DeFi', 'DEX'),
    'FLUID (INSTADAPP)': ('DeFi', 'Lending/Automation'),
    'CAMELOT': ('DeFi', 'DEX'),
    'AERODROME FINANCE': ('DeFi', 'DEX'),
    'PHOENIX': ('DeFi', 'DEX'),
    'VELODROME FINANCE': ('DeFi', 'DEX'),
    'COW PROTOCOL': ('DeFi', 'Aggregator'),
    'PARASWAP': ('DeFi', 'Aggregator'),
    'CURVE.FI': ('DeFi', 'DEX'),

    # Non-DeFi
    'BINANCE': ('Non-DeFi', 'CEX'),
    'METAMASK': ('Non-DeFi', 'Wallet'),
    '42515': ('Unknown', 'Unknown'),
    'WINTERMUTE': ('Internal', 'Internal'),
    'RIZZOLVER (WINTERMUTE)': ('Internal', 'Internal'),
}

def classify(cp):
    cp = cp.strip().upper()
    if cp in mapping:
        return mapping[cp]
    # fallback heuristics
    if 'SWAP' in cp or 'FINANCE' in cp or 'POOL' in cp:
        return ('DeFi', 'DEX')
    return ('Unknown', 'Unknown')

df[['category','subcategory']] = df['counterparty'].apply(lambda x: pd.Series(classify(x)))

df[['counterparty','category','subcategory']].head(20)


,counterparty,category,subcategory
0,CAMELOT,DeFi,DEX
1,UNISWAP,DeFi,DEX
2,AERODROME FINANCE,DeFi,DEX
3,PANCAKESWAP,DeFi,DEX
4,UNISWAP,DeFi,DEX
5,UNISWAP,DeFi,DEX
6,CAMELOT,DeFi,DEX
7,PANCAKESWAP,DeFi,DEX
8,UNISWAP,DeFi,DEX
9,PANCAKESWAP,DeFi,DEX


In [10]:
# Extract DeFi-only interactions

df_defi = df[df['category'] == 'DeFi'].copy()
df_defi.head()


,timestamp,from_address,from_entity,to_address,to_entity,token,value,usd,tx_hash,chain,block_number,counterparty,category,subcategory
0,2025-04-08 12:49:57+00:00,0xB1026b8e7276e7AC75410F1fcbbe21796e8f7526,CAMELOT,0x51C72848c68a965f66FA7a88855F9f7784502a7F,WINTERMUTE,USDC,438.108563,438.108563,0x98058c529466064c355bdfea3cfebe399344678f20bb...,arbitrum_one,324218955.0,CAMELOT,DeFi,DEX
1,2025-04-08 12:49:54+00:00,0xcDa53B1F66614552F834cEeF361A8D12a0B8DaD8,UNISWAP,0x51C72848c68a965f66FA7a88855F9f7784502a7F,WINTERMUTE,ARB,3837.138536,1062.879700,0x405b146ac6a52b93e5a77760a4b49a87015b5c5e0659...,arbitrum_one,324218945.0,UNISWAP,DeFi,DEX
2,2025-04-08 12:49:51+00:00,0xb2cc224c1c9feE385f8ad6a55b4d94E92359DC59,AERODROME FINANCE,0x51C72848c68a965f66FA7a88855F9f7784502a7F,WINTERMUTE,WETH,7.345922,11533.612401,0x7f370fc61bed7241d846c4907a4e616e69ce60ff0192...,base,28663622.0,AERODROME FINANCE,DeFi,DEX
3,2025-04-08 12:49:51+00:00,0x51C72848c68a965f66FA7a88855F9f7784502a7F,WINTERMUTE,0x72AB388E2E2F6FaceF59E3C3FA2C4E29011c2D38,PANCAKESWAP,USDC,2143.712587,2143.712587,0x88b4fb948e95cc2b9a22c48efe3d6a6788209302e72f...,base,28663622.0,PANCAKESWAP,DeFi,DEX
4,2025-04-08 12:49:50+00:00,0x641C00A822e8b671738d32a431a4Fb6074E5c79d,UNISWAP,0x51C72848c68a965f66FA7a88855F9f7784502a7F,WINTERMUTE,WETH,0.470011,737.550466,0x39fc603c882c2f5af51b7f8cfc8eb58753f5e42eff94...,arbitrum_one,324218918.0,UNISWAP,DeFi,DEX
